### Import necessary module

In [1]:
!apt autoremove
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libnvidia-common-510
0 upgraded, 0 newly installed, 1 to remove and 21 not upgraded.
After this operation, 35.8 kB disk space will be freed.
(Reading database ... 128126 files and directories currently installed.)
Removing libnvidia-common-510 (510.108.03-0ubuntu1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=5fb0a0dedeff5cb42032c5df9c11c6d383e45015ba225c3a7ca0e93b0be0e769
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pysp

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1L6pCQkldvdBoaEhRFzL0VnrggEFvqON4'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('Bombing_Operations.json.gz')

id='14dyBmcTBA32uXPxDbqr0bFDIzGxMTWwl'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('Aircraft_Glossary.json.gz')  

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
spark

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2023-02-16 12:38:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  3.46MB/s    in 5.0s    

2023-02-16 12:38:35 (2.65 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### loading dataset

In [8]:
Bombing_Operations = spark.read.json('Bombing_Operations.json.gz')

In [9]:
Aircraft_Glossary = spark.read.json('Aircraft_Glossary.json.gz')

In [10]:
# Showing the dataset formated
Bombing_Operations.show()


+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|AirCraft| ContryFlyingMission|MissionDate|OperationSupported|PeriodOfDay|TakeoffLocation|TargetCountry|TimeOnTarget|          WeaponType|WeaponsLoadedWeight|
+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|              null|          D|   TAN SON NHUT|     CAMBODIA|      1005.0|                null|                  0|
|   EC-47|UNITED STATES OF ...| 1972-12-26|              null|          D|  NAKHON PHANOM|SOUTH VIETNAM|       530.0|                null|                  0|
|    RF-4|UNITED STATES OF ...| 1973-07-28|              null|          D|       UDORN AB|         LAOS|       730.0|                null|                  0|
|     A-1|UNITED STATES OF ...| 1970-02-02|   

In [11]:
# Show Unformated
Bombing_Operations.take(5)

[Row(AirCraft='EC-47', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1971-06-05', OperationSupported=None, PeriodOfDay='D', TakeoffLocation='TAN SON NHUT', TargetCountry='CAMBODIA', TimeOnTarget=1005.0, WeaponType=None, WeaponsLoadedWeight=0),
 Row(AirCraft='EC-47', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1972-12-26', OperationSupported=None, PeriodOfDay='D', TakeoffLocation='NAKHON PHANOM', TargetCountry='SOUTH VIETNAM', TimeOnTarget=530.0, WeaponType=None, WeaponsLoadedWeight=0),
 Row(AirCraft='RF-4', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1973-07-28', OperationSupported=None, PeriodOfDay='D', TakeoffLocation='UDORN AB', TargetCountry='LAOS', TimeOnTarget=730.0, WeaponType=None, WeaponsLoadedWeight=0),
 Row(AirCraft='A-1', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1970-02-02', OperationSupported=None, PeriodOfDay='N', TakeoffLocation='NAKHON PHANOM', TargetCountry='LAOS', TimeOnTarget=1415.0, WeaponType='BLU

In [13]:
#Showing the schma of the JSON file
Bombing_Operations.printSchema()

root
 |-- AirCraft: string (nullable = true)
 |-- ContryFlyingMission: string (nullable = true)
 |-- MissionDate: string (nullable = true)
 |-- OperationSupported: string (nullable = true)
 |-- PeriodOfDay: string (nullable = true)
 |-- TakeoffLocation: string (nullable = true)
 |-- TargetCountry: string (nullable = true)
 |-- TimeOnTarget: double (nullable = true)
 |-- WeaponType: string (nullable = true)
 |-- WeaponsLoadedWeight: long (nullable = true)



In [14]:
Aircraft_Glossary.show()

+--------+--------------------+--------------------+
|AirCraft|        AirCraftName|        AirCraftType|
+--------+--------------------+--------------------+
|     A-1|Douglas A-1 Skyra...|         Fighter Jet|
|    A-26|Douglas A-26 Invader|        Light Bomber|
|    A-37|Cessna A-37 Drago...|Light ground-atta...|
|     A-4|McDonnell Douglas...|         Fighter Jet|
|     A-5|North American A-...|          Bomber Jet|
|     A-6|Grumman A-6 Intruder|     Attack Aircraft|
|     A-7|  LTV A-7 Corsair II|     Attack Aircraft|
|  AC-119|Fairchild AC-119 ...|Military Transpor...|
|  AC-123|Fairchild C-123 P...|Military Transpor...|
|  AC-130|Lockheed AC-130 S...|Fixed wing ground...|
|   AC-47|Douglas AC-47 Spooky|Ground attack air...|
|    AH-1| Bell AH-1 HueyCobra|          Helicopter|
|     B-1| Rockwell B-1 Lancer|Heavy strategic b...|
|    B-52| B-52 Stratofortress|    Strategic bomber|
|    B-57|Martin B-57 Canberra|     Tactical Bomber|
|    B-66|Douglas B-66 Dest...|        Light B

In [15]:
Aircraft_Glossary.take(5)

[Row(AirCraft='A-1', AirCraftName='Douglas A-1 Skyraider', AirCraftType='Fighter Jet'),
 Row(AirCraft='A-26', AirCraftName='Douglas A-26 Invader', AirCraftType='Light Bomber'),
 Row(AirCraft='A-37', AirCraftName='Cessna A-37 Dragonfly', AirCraftType='Light ground-attack aircraft'),
 Row(AirCraft='A-4', AirCraftName='McDonnell Douglas A-4 Skyhawk', AirCraftType='Fighter Jet'),
 Row(AirCraft='A-5', AirCraftName='North American A-5 Vigilante', AirCraftType='Bomber Jet')]

In [16]:
Aircraft_Glossary.printSchema()

root
 |-- AirCraft: string (nullable = true)
 |-- AirCraftName: string (nullable = true)
 |-- AirCraftType: string (nullable = true)



### Executing Queries -select function



In [17]:
Bombing_Operations.select('AirCraft','ContryFlyingMission', 'MissionDate').show()

+--------+--------------------+-----------+
|AirCraft| ContryFlyingMission|MissionDate|
+--------+--------------------+-----------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|
|   EC-47|UNITED STATES OF ...| 1972-12-26|
|    RF-4|UNITED STATES OF ...| 1973-07-28|
|     A-1|UNITED STATES OF ...| 1970-02-02|
|    A-37|     VIETNAM (SOUTH)| 1970-10-08|
|     F-4|UNITED STATES OF ...| 1970-11-25|
|     A-4|UNITED STATES OF ...| 1972-03-08|
|     F-4|UNITED STATES OF ...| 1971-12-27|
|     A-7|UNITED STATES OF ...| 1972-05-24|
|   EC-47|UNITED STATES OF ...| 1972-09-12|
|   CH-53|UNITED STATES OF ...| 1974-06-13|
|   CH-53|UNITED STATES OF ...| 1974-12-19|
|     O-1|     VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|     VIETNAM (SOUTH)| 1974-03-19|
|     C-7|UNITED STATES OF ...| 1970-05-08|
|     A-6|UNITED STATES OF ...| 1971-05-12|
|   EB-66|UNITED STATES OF ...| 1971-12-03|
|    T-28|                LAOS| 1971-12-19|
|     A-6|UNITED STATES OF ...| 1972-08-18|
|     A-7|UNITED STATES OF ...| 

In [18]:
b1 = Bombing_Operations.select('AirCraft','ContryFlyingMission', 'MissionDate')
b1.show()

+--------+--------------------+-----------+
|AirCraft| ContryFlyingMission|MissionDate|
+--------+--------------------+-----------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|
|   EC-47|UNITED STATES OF ...| 1972-12-26|
|    RF-4|UNITED STATES OF ...| 1973-07-28|
|     A-1|UNITED STATES OF ...| 1970-02-02|
|    A-37|     VIETNAM (SOUTH)| 1970-10-08|
|     F-4|UNITED STATES OF ...| 1970-11-25|
|     A-4|UNITED STATES OF ...| 1972-03-08|
|     F-4|UNITED STATES OF ...| 1971-12-27|
|     A-7|UNITED STATES OF ...| 1972-05-24|
|   EC-47|UNITED STATES OF ...| 1972-09-12|
|   CH-53|UNITED STATES OF ...| 1974-06-13|
|   CH-53|UNITED STATES OF ...| 1974-12-19|
|     O-1|     VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|     VIETNAM (SOUTH)| 1974-03-19|
|     C-7|UNITED STATES OF ...| 1970-05-08|
|     A-6|UNITED STATES OF ...| 1971-05-12|
|   EB-66|UNITED STATES OF ...| 1971-12-03|
|    T-28|                LAOS| 1971-12-19|
|     A-6|UNITED STATES OF ...| 1972-08-18|
|     A-7|UNITED STATES OF ...| 

### Executing Queries -where function


In [19]:
b1.where("ContryFlyingMission == 'VIETNAM (SOUTH)'").show()

+--------+-------------------+-----------+
|AirCraft|ContryFlyingMission|MissionDate|
+--------+-------------------+-----------+
|    A-37|    VIETNAM (SOUTH)| 1970-10-08|
|     O-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-03-19|
|   CH-47|    VIETNAM (SOUTH)| 1971-03-29|
|   AC-47|    VIETNAM (SOUTH)| 1971-09-20|
|    UH-1|    VIETNAM (SOUTH)| 1972-08-22|
|    UH-1|    VIETNAM (SOUTH)| 1973-05-13|
|   CH-47|    VIETNAM (SOUTH)| 1975-01-04|
|    A-37|    VIETNAM (SOUTH)| 1971-02-08|
|   C-119|    VIETNAM (SOUTH)| 1970-09-17|
|    A-37|    VIETNAM (SOUTH)| 1971-04-02|
|     A-1|    VIETNAM (SOUTH)| 1972-04-29|
|     A-1|    VIETNAM (SOUTH)| 1972-08-02|
|    UH-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-12-16|
|     A-1|    VIETNAM (SOUTH)| 1970-04-27|
|    UH-1|    VIETNAM (SOUTH)| 1975-02-10|
|    A-37|    VIETNAM (SOUTH)| 1971-03-03|
|     O-1|    VIETNAM (SOUTH)| 1970-08-29|
|     A-1|    VIETNAM (SOUTH)| 1971-07-29|
+--------+-

In [20]:
b1.where('ContryFlyingMission == "VIETNAM (SOUTH)" OR ContryFlyingMission == "LAOS"').show()

+--------+-------------------+-----------+
|AirCraft|ContryFlyingMission|MissionDate|
+--------+-------------------+-----------+
|    A-37|    VIETNAM (SOUTH)| 1970-10-08|
|     O-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-03-19|
|    T-28|               LAOS| 1971-12-19|
|   CH-47|    VIETNAM (SOUTH)| 1971-03-29|
|   AC-47|    VIETNAM (SOUTH)| 1971-09-20|
|    UH-1|    VIETNAM (SOUTH)| 1972-08-22|
|    UH-1|    VIETNAM (SOUTH)| 1973-05-13|
|   CH-47|    VIETNAM (SOUTH)| 1975-01-04|
|    A-37|    VIETNAM (SOUTH)| 1971-02-08|
|    T-28|               LAOS| 1972-05-07|
|   C-119|    VIETNAM (SOUTH)| 1970-09-17|
|    A-37|    VIETNAM (SOUTH)| 1971-04-02|
|     A-1|    VIETNAM (SOUTH)| 1972-04-29|
|     A-1|    VIETNAM (SOUTH)| 1972-08-02|
|    UH-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-12-16|
|     A-1|    VIETNAM (SOUTH)| 1970-04-27|
|    UH-1|    VIETNAM (SOUTH)| 1975-02-10|
|    A-37|    VIETNAM (SOUTH)| 1971-03-03|
+--------+-

### Executing Queries in form of SQL

In [22]:
# Creating view
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query = """
          SELECT AirCraft, ContryFlyingMission, MissionDate
          FROM bombing_operation
          WHERE ContryFlyingMission=='VIETNAM (SOUTH)' OR ContryFlyingMission == 'LAOS'
      """

result = spark.sql(query)
result.show()

+--------+-------------------+-----------+
|AirCraft|ContryFlyingMission|MissionDate|
+--------+-------------------+-----------+
|    A-37|    VIETNAM (SOUTH)| 1970-10-08|
|     O-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-03-19|
|    T-28|               LAOS| 1971-12-19|
|   CH-47|    VIETNAM (SOUTH)| 1971-03-29|
|   AC-47|    VIETNAM (SOUTH)| 1971-09-20|
|    UH-1|    VIETNAM (SOUTH)| 1972-08-22|
|    UH-1|    VIETNAM (SOUTH)| 1973-05-13|
|   CH-47|    VIETNAM (SOUTH)| 1975-01-04|
|    A-37|    VIETNAM (SOUTH)| 1971-02-08|
|    T-28|               LAOS| 1972-05-07|
|   C-119|    VIETNAM (SOUTH)| 1970-09-17|
|    A-37|    VIETNAM (SOUTH)| 1971-04-02|
|     A-1|    VIETNAM (SOUTH)| 1972-04-29|
|     A-1|    VIETNAM (SOUTH)| 1972-08-02|
|    UH-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-12-16|
|     A-1|    VIETNAM (SOUTH)| 1970-04-27|
|    UH-1|    VIETNAM (SOUTH)| 1975-02-10|
|    A-37|    VIETNAM (SOUTH)| 1971-03-03|
+--------+-

In [23]:
query = """
            SELECT COUNT(*) AS NUM_OF_ROWS FROM bombing_operation
      """
result = spark.sql(query)
result.show()

+-----------+
|NUM_OF_ROWS|
+-----------+
|    4400775|
+-----------+



## LAB TASK

1. Find the Air craft, country flying mission and target country for those missions in which time on target was less than 1000.

In [24]:
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query = """
          SELECT AirCraft, ContryFlyingMission, TargetCountry
          FROM bombing_operation
          WHERE TimeOnTarget < 1000
        """

result = spark.sql(query)
result.show()

+--------+--------------------+-------------+
|AirCraft| ContryFlyingMission|TargetCountry|
+--------+--------------------+-------------+
|   EC-47|UNITED STATES OF ...|SOUTH VIETNAM|
|    RF-4|UNITED STATES OF ...|         LAOS|
|     F-4|UNITED STATES OF ...|         LAOS|
|     F-4|UNITED STATES OF ...|         LAOS|
|     A-7|UNITED STATES OF ...|NORTH VIETNAM|
|   EC-47|UNITED STATES OF ...|SOUTH VIETNAM|
|   CH-53|UNITED STATES OF ...|     THAILAND|
|     O-1|     VIETNAM (SOUTH)|SOUTH VIETNAM|
|    UH-1|     VIETNAM (SOUTH)|SOUTH VIETNAM|
|     C-7|UNITED STATES OF ...|SOUTH VIETNAM|
|    T-28|                LAOS|         LAOS|
|     A-6|UNITED STATES OF ...|NORTH VIETNAM|
|     A-7|UNITED STATES OF ...|NORTH VIETNAM|
|   C-123|UNITED STATES OF ...|SOUTH VIETNAM|
|   CH-47|     VIETNAM (SOUTH)|SOUTH VIETNAM|
|     F-4|UNITED STATES OF ...|SOUTH VIETNAM|
|   AU-24|       KOREA (SOUTH)|     CAMBODIA|
|  AC-130|UNITED STATES OF ...|     CAMBODIA|
|    A-37|UNITED STATES OF ...|   

2. Find the target country and the number of times they have been attacked. Show the result in descending order.

In [26]:
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query = """
          SELECT TargetCountry, COUNT(TargetCountry) AS Attack
          FROM bombing_operation
          GROUP BY TargetCountry 
          ORDER BY COUNT(TargetCountry) DESC
       """

result = spark.sql(query)
result.show()

+--------------+-------+
| TargetCountry| Attack|
+--------------+-------+
| SOUTH VIETNAM|2172403|
|          LAOS|1254695|
| NORTH VIETNAM| 694269|
|      CAMBODIA| 218788|
|      THAILAND|  60486|
|WESTPAC WATERS|     74|
|       UNKNOWN|     59|
|   PHILLIPINES|      1|
+--------------+-------+



3. Find the target country which had been attacked most.

In [47]:
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query1 = """
          SELECT TargetCountry, COUNT(TargetCountry) AS NoOfAttack
          FROM bombing_operation
          GROUP BY TargetCountry 
          ORDER BY COUNT(TargetCountry) DESC
       """

result = spark.sql(query1)
result.createOrReplaceTempView('TempResult')

query2 = """
          SELECT *
          FROM TempResult
          WHERE NoOfAttack = (SELECT MAX(NoOfAttack) 
                              FROM TempResult)
       """

result = spark.sql(query2)
result.show()

+-------------+----------+
|TargetCountry|NoOfAttack|
+-------------+----------+
|SOUTH VIETNAM|   2172403|
+-------------+----------+



4. Find the country flying mission which attacked most.

In [48]:
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query1 = """
          SELECT ContryFlyingMission, COUNT(ContryFlyingMission) AS NoOfFlyingMission
          FROM bombing_operation
          GROUP BY ContryFlyingMission 
          ORDER BY COUNT(ContryFlyingMission) DESC
       """

result = spark.sql(query1)
result.createOrReplaceTempView('TempResult')

query2 = """
          SELECT *
          FROM TempResult
          WHERE NoOfFlyingMission = (SELECT MAX(NoOfFlyingMission) 
                                     FROM TempResult)
       """

result = spark.sql(query2)
result.show()

+--------------------+-----------------+
| ContryFlyingMission|NoOfFlyingMission|
+--------------------+-----------------+
|UNITED STATES OF ...|          3708997|
+--------------------+-----------------+



5. Find the take off location and how many times they had used.


In [39]:
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query = """
          SELECT TakeoffLocation, COUNT(TakeoffLocation) AS NoOfTakeoffLocation
          FROM bombing_operation
          GROUP BY TakeoffLocation 
       """

result = spark.sql(query)
result.show()

+--------------------+-------------------+
|     TakeoffLocation|NoOfTakeoffLocation|
+--------------------+-------------------+
|        LOUANG PHRAB|               5164|
|        TAN SON NHUT|             311872|
|            KHE SANH|                 27|
|           NHA TRANG|              56818|
|          WESTPAC 59|                481|
|          KADENA AFB|               5413|
|         TONKIN GULF|             194191|
|           VINH BINH|                 10|
|           AN SON NH|                  3|
|           LONG TIEN|               9499|
|  CORAL SEA (CVA-43)|               1338|
|          WESTPAC 66|              26003|
|          WESTPAC 65|              24407|
|             U TAPAO|              85041|
|          WESTPAC NH|                  5|
|                null|                  0|
|   ORISKANY (CVA-34)|               8793|
|            QUI NHON|               2386|
|           QUANG TRI|                447|
|FRANKLIN D ROOSEV...|               2001|
+----------

6. Find the number of fighter jets from Aircraft_Glossary.


In [40]:
Aircraft_Glossary.createOrReplaceTempView('aircraft_glossary')

query = """
            SELECT COUNT(AirCraftType) as TotalNoOfFighterJet
            FROM aircraft_glossary
            WHERE AirCraftType == 'Fighter Jet'
          """

result = spark.sql(query)
result.show()

+-------------------+
|TotalNoOfFighterJet|
+-------------------+
|                 10|
+-------------------+



7. Find the number of different types air craft for each air craft type.


In [41]:
Aircraft_Glossary.createOrReplaceTempView('aircraft_glossary')

query = """
            SELECT AirCraftType, COUNT(AirCraftType) AS NoOfAirCraftType
            FROM aircraft_glossary
            GROUP BY AirCraftType
          """

result = spark.sql(query)
result.show()

+--------------------+----------------+
|        AirCraftType|NoOfAirCraftType|
+--------------------+----------------+
|Carrier-based Fig...|               1|
|  Utility Prop plane|               1|
|Early Warning air...|               2|
|Military Transpor...|              11|
|Heavy strategic b...|               1|
|Airliner/transpor...|               3|
|        Light Bomber|               3|
|  Transport Aircraft|               1|
|    Cargo Prop plane|               1|
| Airliner/ Transport|               1|
|Airborne early wa...|               1|
|  Jet Fighter Bomber|               1|
|Weather reconnais...|               1|
|Light Observation...|               1|
|Fixed wing ground...|               1|
|        Fighter jet |               1|
|Photographic reco...|               1|
| Light Utility Plane|               3|
|Close air support...|               1|
|     Tactical Bomber|               1|
+--------------------+----------------+
only showing top 20 rows



8. Find the air craft name which had used most in attacks.

In [50]:
Aircraft_Glossary.createOrReplaceTempView('aircraft_glossary')
Bombing_Operations.createOrReplaceTempView('bombing_operation')

query1= """
        SELECT AirCraftName, TargetCountry
        FROM aircraft_glossary A, bombing_operation B
        WHERE A.AirCraft = B.AirCraft
        """
result = spark.sql(query1)

result.createOrReplaceTempView("TempResult1")

query2 = """
            SELECT AirCraftName, COUNT(AirCraftName) as NoOfAirCraftName
            FROM TempResult1
            GROUP BY AirCraftName
          """
result = spark.sql(query2)

result.createOrReplaceTempView("TempResult2")

query3 = """
            SELECT *
            FROM TempResult2
            WHERE NoOfAirCraftName == (SELECT MAX(NoOfAirCraftName) 
                                      FROM TempResult2 )
          """
result = spark.sql(query3)
result.show()

+--------------------+----------------+
|        AirCraftName|NoOfAirCraftName|
+--------------------+----------------+
|McDonnell Douglas...|          909362|
+--------------------+----------------+

